In [140]:
%load_ext nb_mypy
%nb_mypy

State: On DebugOff


The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy


In [154]:
import re
from typing import Set, List, Tuple, Match,Dict
from operator import itemgetter
import copy

# FILENAME : str ="test.txt"
FILENAME : str ="input.txt"
DIAG : bool = False


In [142]:
mmm = re.findall(r"\b(\d+)\b","4  5  7")
mmm

['4', '5', '7']

In [143]:
def getBuckets(lines : List[str]) -> Dict[str,List[str]]:
    pattern : str = r"\b(\d+)\b"
    buckets: List[str] = re.findall(pattern,lines[0])
        
    filledBuckets : Dict[str,List[str]] = {}
    
    if DIAG: print("Buckets:",buckets)
    if DIAG: print("Lines:\n",lines[1:])
    
    for line in lines[1:]:
        for i in range(1,len(buckets)+1):
            ind : int = i*4-3
            
            if i > len(line): break
            
            sym : str = line[ind]
            if sym != " ":
                ll = filledBuckets.setdefault(str(i),[])
                ll.append(sym)
        
    
    return filledBuckets

In [144]:
getBuckets([" 1   2   3   4   5   6   7   8   9 ","[Z] [P] [S] [F] [F] [T] [N] [P] [W]"])

{'1': ['Z'],
 '2': ['P'],
 '3': ['S'],
 '4': ['F'],
 '5': ['F'],
 '6': ['T'],
 '7': ['N'],
 '8': ['P'],
 '9': ['W']}

In [145]:
def getMove(line:str) -> Tuple[int,str,str]:
    pattern : str = "^move (\d+) from (\d+) to (\d+)$"
    mmm = re.search(pattern,line)
    if mmm is not None:
        move_quantity = int(mmm[1])
        move_from = mmm[2]
        move_to = mmm[3]
    else:
        move_quantity = 0
        move_from = ""
        move_to = ""
    return(move_quantity,move_from,move_to)

def getAllMoves(lines:List[str]) -> List[Tuple[int,str,str]]:
    return list(map(getMove,lines))

In [146]:
assert getMove("move 1 from 2 to 1") == (1,"2","1")

assert getMove("move 113 from 22 to 1001") == (113,"22","1001")
assert getAllMoves(["move 1 from 2 to 1","move 113 from 22 to 1001"])==[(1,"2","1"),(113,"22","1001")]

In [147]:
def processFile(file_name : str) -> Tuple[Dict[str,List[str]],List[Tuple[int,str,str]]]:
    with open(file_name) as file:
        lines = file.readlines()
    
    lines_trimmed : List[str] = list(map(lambda x:x.replace("\n",""),lines))
    
    
    line_lim  : int= lines_trimmed.index("")
    
    buckets=getBuckets(lines_trimmed[:line_lim][::-1])
    moves = getAllMoves(lines_trimmed[line_lim+1:])
        
    return buckets,moves
    

In [148]:
def applyMove(buckets : Dict[str,List[str]], move : Tuple[int,str,str], oneByOne:bool) -> None:
    how_many, source,target = move
    
    ll_source = buckets[source]
    ll_target = buckets[target]
    if oneByOne:
        ll_target+=ll_source[-how_many:][::-1]
    else:
        ll_target+=ll_source[-how_many:]
    buckets[source] = ll_source[:-how_many]
    

def applyMovesOnBuckets(buckets,moves,oneByOne:bool) ->Dict[str,List[str]] :
    move : Tuple[int,str,str]
    
    for move in moves: applyMove(buckets,move,oneByOne)
        
    return buckets

In [149]:
def getTopWord(buckets:Dict[str,List[str]]) -> str:
    res = ""
    for i in range(1,len(buckets)+1):
        res += buckets[str(i)][-1]
    return res

In [157]:
buckets : Dict[str,List[str]]
moves : List[Tuple[int,str,str]]
    
buckets,moves = processFile(FILENAME)

bucketsA=copy.deepcopy(buckets)
applyMovesOnBuckets(bucketsA,moves,True)
resA = getTopWord(bucketsA)
print("Result partA:",resA)

bucketsB=copy.deepcopy(buckets)
applyMovesOnBuckets(bucketsB,moves,False)
resB = getTopWord(bucketsB)
print("Result partB:",resB)

Result partA: VQZNJMWTR
Result partB: NLCDCLVMQ
